In [ ]:
# THIS notebook is broken and needs refactoring to take only the last part.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf
import pathlib
import os

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
 tf.__version__

In [ ]:
# silence warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# model config

batch_size = 32
img_width = 320
img_height = 320 #img_width * 2
rgb_channels = 3

In [ ]:
data_dir = pathlib.Path('samples/train')

image_count = len(list(data_dir.glob('*/*.jpg'))) + len(list(data_dir.glob('*/*.png'))) + len(list(data_dir.glob('*/*.jpeg')))
image_count

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size
)

In [ ]:
# Introspect the object
print(train_ds.__dict__.keys(), "\n")
print(train_ds.file_paths[0:5], "\n")
print(train_ds.class_names, "\n")

In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size
)

In [ ]:
class_names = train_ds.class_names
print(class_names)

In [ ]:
for image_batch, labels_batch in train_ds:
    print(image_batch.shape)
    print(labels_batch.shape)
    break

In [ ]:
# Configure dataset for better performance

AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
# Normalize colors from 0-255 to 0-1

normalization_layer = layers.Rescaling(1./255)

normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))

# Notice the pixel values are now in `[0,1]`.
first_image = image_batch[0]
print(np.min(first_image), np.max(first_image))

In [ ]:
# Creating the model

num_classes = len(class_names)
input_shape = (img_height, img_width, rgb_channels)

# Add GaussianNoise layer for robustness
noise = layers.GaussianNoise(0.01, input_shape=input_shape)

model = Sequential([
    noise,
  layers.Rescaling(1./255, input_shape=input_shape),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes) # output layer,
])

In [ ]:
type(train_ds)

In [ ]:
# Compile model

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [ ]:
model.build(input_shape=input_shape)
model.summary()

In [ ]:
# Train the model

epochs=10
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

In [ ]:
type(history)

In [ ]:
# Visualizing results

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
# Predict on new data

pred_data_dir = pathlib.Path('samples/test')
pred_images = list(pred_data_dir.glob('**/*.jpg')) + list(pred_data_dir.glob('**/*.png'))

print(pred_images)

In [ ]:
threshold = 80

for im in pred_images:
    img_path_str = os.path.join(os.getcwd(), str(im))
    img_title = os.path.basename(img_path_str)
    #img_path = tf.keras.utils.get_file(os.path.basename(img_title), origin=img_path_str)

    img = tf.keras.utils.load_img(
        img_path_str, target_size=(img_height, img_width)
    )
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch

    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0])

    print(
        img_title,
        "This image most likely belongs to {} with a {:.2f} percent confidence."
        .format(class_names[np.argmax(score)], 100 * np.max(score))
    )

In [ ]:
# Visualize what each layer is doing
# https://www.analyticsvidhya.com/blog/2020/11/tutorial-how-to-visualize-feature-maps-directly-from-cnn-layers/

layer_names = [layer.name for layer in model.layers]
layer_names

In [ ]:
from tensorflow.keras.preprocessing.image import img_to_array, load_img

layer_outputs = [layer.output for layer in model.layers]
visual_model = tf.keras.models.Model(inputs=model.input, outputs=layer_outputs)

image_path= r"samples/test/scr04.png"
img = load_img(image_path, target_size=(img_height, img_width))
img_array = img_to_array(img)    
print(img_array.shape)
#input_img = x.reshape((1,) + x.shape)                   
#input_img /= 255.0

plt.imshow(img_array.astype("uint8"))

In [ ]:
# reshape, to be able to pass it as prediction
input_img = img_array.reshape((1,) + img_array.shape)                   
input_img /= 255.0
print(input_img.shape)

# rescale
input_img = input_img / 255

In [ ]:
# predict

feature_maps = visual_model.predict(input_img)
print(len(feature_maps))

In [ ]:
score2 = tf.nn.softmax(model.predict(input_img)[0])
np.argmax(score2)
print(
    "'{}' belongs to {} with a {:.2f} percent confidence."
    .format(image_path, class_names[np.argmax(score2)], 100 * np.max(score2))
)

In [ ]:
%matplotlib inline

In [ ]:
# plotting the graph
for layer_name, feature_map in zip(layer_names, feature_maps):
    if len(feature_map.shape) == 4: # 4-dimensions layer
        channels = feature_map.shape[-1]
        size_h = feature_map.shape[1]
        size_w = feature_map.shape[2]
        #print(layer_name)
        #print('feature_map.shape:', feature_map.shape)
        
        size = feature_map.shape[1]
        display_grid = np.zeros((size_h, size_w * channels))
        #print('display_grid.shape:', display_grid.shape)
        for i in range(channels):
            x = feature_map[0, :, :, i]
            x -= x.mean()
            x /= x.std()
            x *= img_width
            x += img_width
            x = np.clip(x, 0, 255).astype('uint8')
            # print(x.shape)
            # tile each filter into this big horizontal grid
            display_grid[:, i*size_w: (i + 1) * size_w] = x
        
        #print("\n")
        scale = 20. / channels
        plt.figure(figsize=(scale * channels, scale))
        plt.title(layer_name)
        plt.grid(False)
        plt.imshow(display_grid, aspect='auto', cmap='viridis')
